# 第18章 クラスタリング
## 18.1 k平均法

### データを読み込む

In [ ]:
import polars as pl

wine = pl.read_csv("../data/wine.csv")
display(wine.head(n = 3))
display(wine.describe())

In [ ]:
from sklearn.cluster import KMeans

# クラスタリング
kmeans = KMeans(n_clusters = 3, random_state = 42).fit(wine)
display(kmeans)

# クラス数
import numpy as np
display(np.unique(kmeans.labels_, return_counts = True))

# クラスタリング結果(後で次元削減したDataFrameに結合するためDataFrameで初期化する)
clusters = pl.DataFrame(pl.Series("cluster", kmeans.labels_))
display(clusters.head())

### 18.1 主成分分析で次元を減らして、クラスタリング結果を可視化する

In [ ]:
from sklearn.decomposition import PCA

# 学習
# 答えである品種列("Cultivar")を削除しておく
pca = PCA(n_components = 2).fit(wine.drop("Cultivar"))

# 次元削減
pca_transformed = pca.transform(wine.drop("Cultivar"))
df_transformed = (
    pl
    .DataFrame(pca_transformed)
    .rename(mapping = {"column_0": "pca1", "column_1": "pca2"})
)

# クラスタリング結果を結合する
df_result = pl.concat(items = [df_transformed, clusters], how = "horizontal")
display(df_result.head())

# 可視化
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
sns.scatterplot(
    x = "pca1",
    y = "pca2",
    data = df_result,
    hue = "cluster",
    ax = ax
)
plt.show()


### クラスタリング結果とワイン品種のピボットテーブルで分類精度を確認する

In [ ]:
df_pivot = (
    pl
    .concat(items = [wine, clusters], how = "horizontal")
    .pivot(index = "cluster", columns = "Cultivar", values = "Alcohol", aggregate_function = "count")
)
display(df_pivot)